# Time series analysis  (trend, slope, ... calculation) for JEDI-CUBES

This notebook was set up to help with the analysis of time series in jedi cubes. It can be connected directly to JEDI cubes and examine a time series contained there for various reference units (NUTS3,2,1,0, 1km, 10km, ....).
If you have any questions, please contact: loehnertz@space4environment.com

---------------------------------------------------------------------------------------------------------------------------------------


## (1) connection to jedi cube tables on MS-SQL (TEAL, ONAGER, GREENMONKEY, OSPREY...)
    !Please note: you need read and partially write permissions for the database to be used.!
        
<img src="./images/ms_sql_server_image.png" style="height: 600px;"/>

In [ ]:
## Reading libaries and start a test-connection to different MS-SQL Server:
import pyodbc 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy as sa
from sqlalchemy import create_engine, event
from sqlalchemy.engine.url import URL

        
#### TEAL
print ("connect to engine......")
engine = sa.create_engine('mssql+pyodbc://' + "GREENMONKEY" + '/' + "Carbon_Mapping" + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')

print ("connection to greenmonkey found")
print ("Reading LUCAS topsoil 2018 data from greenmonkey and calc. number of points, min, avg and max OC2018 for every NUTS2 region in Austria:")
query=('''

SELECT  [LEVEL2_COD_v2021]
		,[LULUCF_CODE]
		,[LULUCF_DESCRIPTION]  
		,count([POINTID]) as count_points
		,min([OC]) as min_OC
		,max([OC]) as max_OC
		,avg([OC]) as min_OC
  FROM [Carbon_Mapping].[lucas].[lucas_topsoil_2018_v2021]
  join [LUT].[LUCAS_landcover_into_LULUCF_classification] on [LUCAS_landcover_into_LULUCF_classification].[LUCAS Level2] = [lucas_topsoil_2018_v2021].LC
  where left(LEVEL2_COD_v2021,2)='AT'
  group by 
  [LEVEL2_COD_v2021], [LULUCF_CODE] ,[LULUCF_DESCRIPTION] order by [LEVEL2_COD_v2021], [LULUCF_DESCRIPTION]
''')  
df_oc2018_nuts2_stat_AT = pd.read_sql(query, engine)
print(df_oc2018_nuts2_stat_AT.head(26))
#pivot 

print ("end....")

-------------------------------